<a href="https://colab.research.google.com/github/suyash091/japanese-sentiment-analysis/blob/master/Machine_Learning_(Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pandas as pd

In [2]:
!pip install fugashi[unidic-lite]
!python -m unidic download
!pip install mecab-python3

     |████████████████████████████████| 481kB 4.5MB/s 
     |████████████████████████████████| 47.3MB 65kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.7-cp36-none-any.whl size=47556594 sha256=cb9833098b84dd042edf181d50d683f22cd2290942ee58c6523ef19f90f609e7
  Stored in directory: /root/.cache/pip/wheels/a8/82/7d/086724645e33a575aafd0b1dae2835c37d2c00c6a0a96ee3a0
Successfully built unidic-lite
/usr/bin/python3: No module named unidic
     |████████████████████████████████| 3.5MB 4.5MB/s 


In [3]:
!pip install janome==0.3.4

     |████████████████████████████████| 20.0MB 1.2MB/s 
  Created wheel for janome: filename=Janome-0.3.4-cp36-none-any.whl size=20682717 sha256=47bb5cbd57177e0ef4238d5ae1ec13b18d6b308218e081e0a187733483df1a02
  Stored in directory: /root/.cache/pip/wheels/01/46/bd/74692e48da56b5d85eb6fd0660ae0bc9b28845dd4543b4f8c1
Successfully built janome


In [5]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
from tqdm import tqdm
char_filters = [UnicodeNormalizeCharFilter(), RegexReplaceCharFilter(u'蛇の目', u'janome')]
tokenizer = Tokenizer()
token_filters = [CompoundNounFilter(), POSStopFilter(['記号','助詞']), LowerCaseFilter()]
a = Analyzer(char_filters, tokenizer, token_filters)

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
main=pd.read_csv('/content/drive/My Drive/CompleteTwitterdata.csv')[['Tweet','Sentiment','Label']]

In [17]:
mainalt=pd.read_csv('/content/TwitterReview (8).csv')[['Tweet','Sentiment','Label']]
main=pd.concat([main,mainalt])
main=main.dropna()


In [18]:
len(main)

919290

In [ ]:
del datalist, labelist

In [15]:

import pandas as pd
import requests
import MeCab
import os
url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
response = requests.get(url)
stopwords = [w for w in response.content.decode().split('\r\n') if w != '']

tagger = MeCab.Tagger('/usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
target_parts_of_speech = ('名詞')

In [13]:
def cleantext(text):
  text=' '.join(text)
  return re.sub(' +', ' ', re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text).replace('https://', ''))

In [30]:
def cleantext(text):
  return re.sub(r"http\S+", "", text)

In [32]:
def fetch_tokenize(text):
    tokenized_text = []
    for chunk in [token.surface for token in a.analyze(text)]:
      if chunk not in stopwords:
          tokenized_text.append(chunk.lower())
    return cleantext(' '.join(tokenized_text))


In [33]:
main.head()

,Tweet,Sentiment,Label
0,✅ポイント高還元 #スーパーDEAL https://t.co/7LROwRwpBL ℹ️4...,0.2,Positive
1,@shinyaFK Twitterキャンペーンの結果は…残念 マイナポイントは電子マネーWA...,-0.3,Negative
2,【 8/10(月)先行予約開始 】ニット 長袖 秋 冬 春 レディース プルオーバー トップ...,0.2,Positive
3,@harukanaworld1 Twitterキャンペーンの結果は…残念 マイナポイントは電...,-0.2,Negative
4,300円クーポン進呈ポイント2倍https://t.co/zygGL2iwMOミント香り付ク...,0.4,Positive


In [ ]:
main['body_wakati'] = main.Tweet.apply(fetch_tokenize)

In [ ]:
dataaltseries=[]
for text in tqdm(alttokenlist[:3481]+[alttokenlist[3480]]+alttokenlist[3482:]):
  dataaltseries.append(cleantext(text))

dataseries=[cleantext(text) for text in tokenlist]

dftweets = pd.DataFrame({'tweets':dataseries+dataaltseries})

100%|██████████| 88366/88366 [00:01<00:00, 57045.72it/s]


In [ ]:
from keras.preprocessing.text import Tokenizer
max_fatures = 50000
tokenizer = Tokenizer(num_words=50000,split=' ')
tokenizer.fit_on_texts(dftweets['tweets'].values)
X1 = tokenizer.texts_to_sequences(dftweets['tweets'].values)
X1 = pad_sequences(X1)

NameError: ignored

In [ ]:
import pickle

# saving
#with open('tokenizerscore.pickle', 'wb') as handle:
#    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('/content/drive/My Drive/tokenizerscore.pickle', 'rb') as handle:
    tokenizer0 = pickle.load(handle)
with open('/content/drive/My Drive/X1.pickle', 'rb') as handle:
    x1 = pickle.load(handle)
with open('/content/drive/My Drive/y.pickle', 'rb') as handle:
    y = pickle.load(handle)

In [ ]:
import pickle

# saving
with open('X1.pickle', 'wb') as handle:
    pickle.dump(X1, handle, protocol=pickle.HIGHEST_PROTOCOL)

# saving
with open('y.pickle', 'wb') as handle:
    pickle.dump(y, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
#with open('tokenizerscore.pickle', 'rb') as handle:
#    tokenizer0 = pickle.load(handle)

In [ ]:
len(y)==len(X1)

True

In [ ]:
y=main['Sentiment'].values.tolist()+mainalt['Sentiment'].values.tolist()

NameError: ignored

In [ ]:
max_fatures = len(tokenizer0.word_index) + 1

In [ ]:
max_fatures

770758

In [ ]:
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
embed_dim = 150
lstm_out = 256*2
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X1.shape[1]))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dense(1,activation='linear'))
model.compile(loss = 'mean_absolute_error', optimizer='adam',metrics = ['mean_squared_error'])
print(model.summary())

NameError: ignored

In [ ]:

from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
def define_model(length, vocab_size):
	inputs = Input(shape=(length,))
	# channel 1
	embedding1 = Embedding(vocab_size, 100)(inputs)
	conv1 = Conv1D(filters=32, kernel_size=2, activation='relu')(embedding1)
	drop1 = Dropout(0.25)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	embedding2 = Embedding(vocab_size, 100)(inputs)
	conv2 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding2)
	drop2 = Dropout(0.25)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	embedding3 = Embedding(vocab_size, 100)(inputs)
	conv3 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding3)
	drop3 = Dropout(0.25)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(128, activation='relu')(merged)
	outputs = Dense(1, activation='linear')(dense1)
	model = Model(inputs=[inputs], outputs=outputs)
	# compile
	model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='multichannel.png')
	return model

In [ ]:
X1.shape

(824674, 60)

In [ ]:
import numpy as np
X1=[]
for x in x1:
  X1.append(x.tolist()[:-1])

In [ ]:
X1=np.array(X1)

In [ ]:
X1[0:3]

array([[     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,  11641,    590,    893,    173,
           298,   9355,  13992],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0, 182379,     24,     20,     21,     11,     23,      3,
            22,      8,     16,      5,     10,      7,      9,     17,
             3,     18,     19,

In [ ]:

model = define_model(X1.shape[1], max_fatures)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 59)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 59, 100)      77075800    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 59, 100)      77075800    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 59, 100)      77075800    input_1[0][0]                    
_______________________________________________________________________________________

In [ ]:
ylist=[]
for i in y:
  if i>=0:
    ylist.append(1)
  else: 
    ylist.append(0)
ylist=np.array(ylist)

In [ ]:
batch_size = 1024
import keras
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath='/content/models/sentimentscore.h5',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
model.fit(X1, np.array(y), validation_split = 0.05, epochs = 50, batch_size=batch_size, verbose = 1)

Epoch 1/50
766/766 [==============================] - 1572s 2s/step - loss: 0.1237 - mean_squared_error: 0.0368 - val_loss: 0.1624 - val_mean_squared_error: 0.0483
Epoch 2/50
766/766 [==============================] - 1558s 2s/step - loss: 0.0879 - mean_squared_error: 0.0203 - val_loss: 0.1551 - val_mean_squared_error: 0.0454
Epoch 3/50
766/766 [==============================] - 1550s 2s/step - loss: 0.0711 - mean_squared_error: 0.0140 - val_loss: 0.1536 - val_mean_squared_error: 0.0448
Epoch 4/50
766/766 [==============================] - 1561s 2s/step - loss: 0.0606 - mean_squared_error: 0.0106 - val_loss: 0.1539 - val_mean_squared_error: 0.0453
Epoch 5/50
766/766 [==============================] - 1609s 2s/step - loss: 0.0539 - mean_squared_error: 0.0087 - val_loss: 0.1577 - val_mean_squared_error: 0.0473
Epoch 6/50
526/766 [===================>..........] - ETA: 8:29 - loss: 0.0491 - mean_squared_error: 0.0074

KeyboardInterrupt: ignored

In [ ]:
model.save('lstmmodel.h5')

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X1, np.array(y), test_size=0.2, random_state=42)

In [ ]:
min(y_pred.tolist())

-8268964.5479337545

In [ ]:
from sklearn.linear_model import Ridge, Lasso
lasso001 = Lasso(alpha=0.0001, max_iter=10e5)
lasso001.fit(x_train,y_train)
y_pred=lasso001.predict(x_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

0.15105568508262415

In [ ]:
import pandas as pd

import chardet
import pandas as pd

with open('/content/TestDataForSentimentAnalysisUTF.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

twitterdata1=pd.read_csv('/content/TestDataForSentimentAnalysisUTFHeader.csv',encoding = result['encoding'],error_bad_lines=False)


In [ ]:
#twitterdata1=twitterdata1[['Tweet','Sentiment','Label']]
#twitterdata1=twitterdata1.dropna()
twitterdata1.head(3)

,Tweet
0,楽天の自社グループも厳しい状況の様なので、米配車サービス大手リフトよりもソッチに集中するのは...
1,#楽天#詐欺メール 楽天市場から支払い情報が一致してないってメール来た酔ってる時に間違って入...
2,先月の8日から始めた楽天Room、Aランクになってた✨✨欲しいものありすぎて困る。あまむら食...


In [ ]:
testlist=twitterdata1.Tweet.values.tolist()

In [ ]:
testtokenlist=[]
for i in tqdm(testlist):
  testtokenlist.append([token.surface for token in a.analyze(i)])

100%|██████████| 6954/6954 [00:44<00:00, 155.88it/s]


In [ ]:
Sentiments=twitterdata1.Sentiment.values.tolist()

In [ ]:
datatestseries=[]
for text in tqdm(testtokenlist):
  datatestseries.append(cleantext(text))

dftesttweets = pd.DataFrame({'tweets':datatestseries})

100%|██████████| 6954/6954 [00:00<00:00, 77527.78it/s]


In [ ]:
datatestseries

['5 つく 日 日田天領水 まとめ買い 安い 店 順に 日田天領水 12l×2箱楽天市場 天然活性水素水 [楽天] ',
 'マラソン 終わっ た また 来る でも 今 欲しい ! もの 売り切れる 前 買い まっしょう❤ 私 服 欲しい ... 楽天市場 [楽天] #rakuafl',
 '頂い た いい リツイート 数 皆さん 呉 愛 感じ まし た (* ▽`*)お世話 なっ いる 人 贈り物 ちょっとした 自分 ご 褒美 良い 思い ます 釜めし系以外 色々 選べ ます ♪... ',
 'レビュー 欲しい 人 必見!レビュー 書ける 人 必見!【ガチンコ モニターズ ━━━ !!!!!★amazon 楽天市場 ヤフーショッピング#レビュー募集',
 'レビュー 欲しい 人 必見!レビュー 書ける 人 必見!【ガチンコ モニターズ ━━━ !!!!!★amazon 楽天市場 ヤフーショッピング#レビュー募集',
 'レビュー 欲しい 人 必見!レビュー 書ける 人 必見!【ガチンコ モニターズ ━━━ !!!!!★amazon 楽天市場 ヤフーショッピング#レビュー募集',
 '楽天市場 安心保証付き中古パソコン ガイドブック&office付 初心者 すぐ 使える !初期設定不要 ノートパソコン 中 ... ベストセラー! ',
 'nhk 快眠 良い 紹介 さ れ た そう 楽天市場 楽天ランキング1位入賞!】 cloud b クラウドビー アクアタートル 睡眠tranquil turtle 子守唄 ぬいぐるみ 出産祝い [楽天]... ',
 '北海道 一年 一番 美味しい 時期 獲 れ た 本 いくら 薄皮イクラ 醤油漬け 80g 軍艦 約 10個分 秋鮭卵 魚 お 取り寄せ グルメ 高級海鮮 一人前 お うち時間 国産 食べ物 お 試し 防腐剤等 無 添加 復興(ふっこう) お うち時間応... ',
 'おはよう ござい ます 本日 ブログ更新 です ✊✊✊ここ数日 楽天市場 spu 取り上げる 事 多かっ た 今日 です 9種類 あっ 値段 安い 楽天カード 超 かんたん保険 多く 人 spu 還元分 得 できる 思い ます ... ',
 '== (● ∀`) ┃楽天市場ドア┃- コスプレ♪価格:710円 !?80dタイツ単品 コスプレ 可愛

In [ ]:
Xtest1 = tokenizer0.texts_to_sequences(dftesttweets['tweets'].values)
Xtest1 = pad_sequences(Xtest1,maxlen=59)

In [ ]:
Xtest1.shape


(6954, 59)

In [ ]:
y_pred=model.predict(X1[:20000])

In [ ]:
import numpy as np
Y1_test=np.array(y[:20000])

In [ ]:
np.array(Sentiments)

array([ 0.4,  0.4,  0.2, ..., -0.7, -0.5, -0.1])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_pred.ravel(), Y1_test)

0.04886331252975739

In [ ]:
print(y_pred[:100])
print(Y1_test[0:100])

[-0.  -0.   0.8  0.4  0.4  0.4  0.4  0.2  0.3  0.3  0.2  0.6 -0.   0.4
  0.4  0.4  0.4  0.4  0.1  0.7  0.2  0.5 -0.   0.8  0.8  0.4  0.4 -0.
  0.2  0.1  0.2  0.8  0.2  0.9  0.4  0.8  0.8  0.9  0.6  0.5  0.1  0.4
  0.5  0.3  0.2  0.1 -0.   0.5  0.6  0.6  0.1  0.2  0.5  0.9  0.6  0.6
 -0.   0.5  0.4  0.1  0.1  0.7  0.8  0.4  0.3  0.6  0.1  0.6  0.1  0.2
  0.7  0.4  0.4  0.4  0.4  0.2  0.5  0.1  0.3 -0.   0.2  0.2  0.1  0.2
  0.3  0.4  0.1  0.1  0.   0.2  0.2  0.1  0.1  0.5  0.6  0.4  0.4  0.4
 -0.   0.3]
[0.  0.  0.8 0.4 0.4 0.4 0.4 0.2 0.3 0.3 0.2 0.6 0.  0.4 0.4 0.4 0.4 0.4
 0.2 0.6 0.2 0.4 0.  0.9 0.8 0.4 0.4 0.  0.2 0.1 0.2 0.8 0.4 0.9 0.4 0.9
 0.8 0.9 0.6 0.5 0.  0.4 0.5 0.  0.2 0.  0.  0.5 0.6 0.5 0.6 0.4 0.5 0.9
 0.7 0.6 0.  0.5 0.4 0.  0.1 0.7 0.8 0.4 0.8 0.8 0.1 0.5 0.1 0.4 0.8 0.4
 0.4 0.  0.4 0.2 0.8 0.1 0.3 0.  0.  0.  0.  0.4 0.  0.4 0.1 0.  0.1 0.3
 0.9 0.1 0.1 0.4 0.5 0.4 0.4 0.3 0.  0. ]


In [ ]:
import numpy as np
y_pred=np.around(y_pred.ravel(), decimals=1)
mean_absolute_error(y_pred.ravel(), Y1_test)

0.06820987776278253

In [ ]:
model.fit(Xtest1, Y1_test, validation_split = 0.1, epochs = 50, batch_size=batch_size, verbose = 1)

Epoch 1/50
12/12 [==============================] - 24s 2s/step - loss: 0.1287 - mean_squared_error: 0.0376 - val_loss: 0.1499 - val_mean_squared_error: 0.0438
Epoch 2/50
12/12 [==============================] - 24s 2s/step - loss: 0.0972 - mean_squared_error: 0.0248 - val_loss: 0.1470 - val_mean_squared_error: 0.0423
Epoch 3/50
12/12 [==============================] - 24s 2s/step - loss: 0.0770 - mean_squared_error: 0.0169 - val_loss: 0.1481 - val_mean_squared_error: 0.0428
Epoch 4/50
 6/12 [==============>...............] - ETA: 10s - loss: 0.0664 - mean_squared_error: 0.0135

KeyboardInterrupt: ignored

In [ ]:
mean_absolute_error(y_pred.ravel(), Y1_test)

0.0815298311112923

In [ ]:
import numpy as np
y_pred=np.around(y_pred.ravel(), decimals=1)
mean_absolute_error(y_pred.ravel(), Y1_test)

ValueError: ignored

In [ ]:
pd.DataFrame({'tweets':datatestseries}).to_csv('tweets.csv')
pd.DataFrame({'original_results':Y1_test.tolist()}).to_csv('original_results.csv')
pd.DataFrame({'model_results':y_pred.ravel().tolist()}).to_csv('model_results.csv')

In [ ]:
twitterdata1['Sentiments']=y_pred.tolist()

In [ ]:
label=[]
for tweet in twitterdata1['Sentiments'].values.tolist():
  if tweet>=0:
    label.append('Positive')
  else:
    label.append('Negative')

In [ ]:
twitterdata1['Label']=label

In [ ]:
twitterdata1.tail(50)

,Tweet,Sentiments,Label
6904,お得なクーポンがほしい！ #楽天スーパーSALEでほしいもの https://t.co/Hs...,0.1,Positive
6905,愛猫用のサイエンスダイエットシニア用180cmのキッチンマットフェイスマスクデオナチュレのソ...,0.1,Positive
6906,お得なクーポンがほしい！ #楽天スーパーSALEでほしいもの https://t.co/nc...,0.4,Positive
6907,今回も買うぞー！お得なクーポンがほしい！ #楽天スーパーSALEでほしいもの https:/...,0.2,Positive
6908,お得なクーポンがほしい！ #楽天スーパーSALEでほしいもの https://t.co/EV...,0.1,Positive
6909,お得なクーポンがほしい！ #楽天スーパーSALEでほしいもの https://t.co/UZ...,0.1,Positive
6910,お得なクーポンがほしい！ #楽天スーパーSALEでほしいもの https://t.co/um...,0.1,Positive
6911,高性能なワイヤレスイヤホン、中華製のへっぽこなのはもうこりごりっ！お得なクーポンがほしい！ ...,0.0,Positive
6912,#ダイソン掃除機 お得なクーポンがほしい！ #楽天スーパーSALEでほしいもの https:...,0.1,Positive
6913,お得なクーポンがほしい！ #楽天スーパーSALEでほしいもの https://t.co/yb...,0.1,Positive


In [ ]:
twitterdata1.to_csv('CNNAnalysis0.csv',encoding=result['encoding'])

In [ ]:
twitterdata1.Label.value_counts()

Negative    3508
Positive    3446
Name: Label, dtype: int64